In [65]:
import torch
import torchvision
from torchvision import transforms, datasets
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import math
import h5py
import pickle

### Define ResNet

In [34]:
## Basic ResNet model

def init_layer(L):
    # Initialization using fan-in
    if isinstance(L, nn.Conv2d):
        n = L.kernel_size[0]*L.kernel_size[1]*L.out_channels
        L.weight.data.normal_(0,math.sqrt(2.0/float(n)))
    elif isinstance(L, nn.BatchNorm2d):
        L.weight.data.fill_(1)
        L.bias.data.fill_(0)




# Simple ResNet Block
class SimpleBlock(nn.Module):
    def __init__(self, indim, outdim, half_res):
        super(SimpleBlock, self).__init__()
        self.indim = indim
        self.outdim = outdim
        self.C1 = nn.Conv2d(indim, outdim, kernel_size=3, stride=2 if half_res else 1, padding=1, bias=False)
        self.relu1 = nn.ReLU(inplace=True)
        self.relu2 = nn.ReLU(inplace=True)
        self.BN1 = nn.BatchNorm2d(outdim)
        self.C2 = nn.Conv2d(outdim, outdim,kernel_size=3, padding=1,bias=False)
        self.BN2 = nn.BatchNorm2d(outdim)

        self.parametrized_layers = [self.C1, self.C2, self.BN1, self.BN2]

        self.half_res = half_res

        # if the input number of channels is not equal to the output, then need a 1x1 convolution
        if indim!=outdim:
            self.shortcut = nn.Conv2d(indim, outdim, 1, 2 if half_res else 1, bias=False)
            self.parametrized_layers.append(self.shortcut)
            self.BNshortcut = nn.BatchNorm2d(outdim)
            self.parametrized_layers.append(self.BNshortcut)
            self.shortcut_type = '1x1'
        else:
            self.shortcut_type = 'identity'

        for layer in self.parametrized_layers:
            init_layer(layer)

    def forward(self, x):
        out = self.C1(x)
        out = self.BN1(out)
        out = self.relu1(out)
        out = self.C2(out)
        out = self.BN2(out)
        short_out = x if self.shortcut_type == 'identity' else self.BNshortcut(self.shortcut(x))
        out = out + short_out
        out = self.relu2(out)
        return out



# Bottleneck block
class BottleneckBlock(nn.Module):
    def __init__(self, indim, outdim, half_res):
        super(BottleneckBlock, self).__init__()
        bottleneckdim = int(outdim/4)
        self.indim = indim
        self.outdim = outdim
        self.C1 = nn.Conv2d(indim, bottleneckdim, kernel_size=1,  bias=False)
        self.relu = nn.ReLU()
        self.BN1 = nn.BatchNorm2d(bottleneckdim)
        self.C2 = nn.Conv2d(bottleneckdim, bottleneckdim, kernel_size=3, stride=2 if half_res else 1,padding=1)
        self.BN2 = nn.BatchNorm2d(bottleneckdim)
        self.C3 = nn.Conv2d(bottleneckdim, outdim, kernel_size=1, bias=False)
        self.BN3 = nn.BatchNorm2d(outdim)

        self.parametrized_layers = [self.C1, self.BN1, self.C2, self.BN2, self.C3, self.BN3]
        self.half_res = half_res


        # if the input number of channels is not equal to the output, then need a 1x1 convolution
        if indim!=outdim:
            self.shortcut = nn.Conv2d(indim, outdim, 1, stride=2 if half_res else 1, bias=False)
            self.parametrized_layers.append(self.shortcut)
            self.shortcut_type = '1x1'
        else:
            self.shortcut_type = 'identity'

        for layer in self.parametrized_layers:
            init_layer(layer)


    def forward(self, x):

        short_out = x if self.shortcut_type == 'identity' else self.shortcut(x)
        out = self.C1(x)
        out = self.BN1(out)
        out = self.relu(out)
        out = self.C2(out)
        out = self.BN2(out)
        out = self.relu(out)
        out = self.C3(out)
        out = self.BN3(out)
        out = out + short_out

        out = self.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self,block,list_of_num_layers, list_of_out_dims, num_classes=1000, only_trunk=False ):
        # list_of_num_layers specifies number of layers in each stage
        # list_of_out_dims specifies number of output channel for each stage
        super(ResNet,self).__init__()
        self.grads = []
        self.fmaps = []
        assert len(list_of_num_layers)==4, 'Can have only four stages'
        conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                                               bias=False)
        bn1 = nn.BatchNorm2d(64)
        relu = nn.ReLU()
        pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        init_layer(conv1)
        init_layer(bn1)


        trunk = [conv1, bn1, relu, pool1]
        indim = 64
        for i in range(4):

            for j in range(list_of_num_layers[i]):
                half_res = (i>=1) and (j==0)
                B = block(indim, list_of_out_dims[i], half_res)
                trunk.append(B)
                indim = list_of_out_dims[i]

        self.only_trunk=only_trunk
        if not only_trunk:
            avgpool = nn.AvgPool2d(7)
            trunk.append(avgpool)

        self.trunk = nn.Sequential(*trunk)
        self.final_feat_dim = indim
        if not only_trunk:
            self.classifier = nn.Linear(indim, num_classes)
            self.classifier.bias.data.fill_(0)

    def forward(self, x):
        out = self.trunk(x)
        if self.only_trunk:
            return out
        out = out.view(out.size(0),-1)
        scores = self.classifier(out)
        return scores, out


def ResNet10(num_classes=1000, only_trunk=False):
    return ResNet(SimpleBlock, [1,1,1,1],[64,128,256,512], num_classes, only_trunk)

In [316]:
data_transform = transforms.Compose([
        transforms.Resize((300,300)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
train_dataset = datasets.ImageFolder(root='../novel-data/train',
                                           transform=data_transform)
train_dataset_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size=20, shuffle=True, 
                                            num_workers=4)
test_dataset = datasets.ImageFolder(root='../novel-data/test',
                                           transform=data_transform)
test_dataset_loader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size=140, shuffle=True,
                                             num_workers=4)
test_x, test_y = next(iter(test_dataset_loader))
test_x, test_y = Variable(test_x), Variable(test_y)

In [318]:
net = ResNet10(14)

In [319]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

In [314]:
def calc_accuracy(mdl, X, Y):
    # TODO: why can't we call .data.numpy() for train_acc as a whole?
    outputs, _ = mdl(X)
    max_vals, max_indices = torch.max(outputs,1)
    train_acc = (max_indices == Y).sum().data.numpy()/max_indices.size()[0]
    return train_acc

### Train ResNet

In [321]:
for epoch in range(10):  # loop over the dataset multiple times
    acc = 0
    running_loss = 0.0
    test_acc = 0
    train_dataset_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size=20, shuffle=True,
                                             num_workers=4)
    
    for i, data in enumerate(train_dataset_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, _ = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        acc += calc_accuracy(net, inputs, labels)
        # print statistics
        running_loss += loss.data[0]
        #if i % 5 == 4:    # print every 2000 mini-batches
            #print('[%d, %5d] loss: %.3f' %
            #      (epoch + 1, i + 1, running_loss / 5))
        #    running_loss = 0.0
        
    # Compute test accuracy
    #test_acc += calc_accuracy(net, test_x, test_y)

    print('Loss: ', running_loss)
    print('Training accuracy: ', acc/(len(train_dataset_loader)))
    #print('Test accuracy: ', test_acc)

print('Finished Training')

Loss:  34.1836621761322
Training accuracy:  [0.62037037]
Loss:  26.584738075733185
Training accuracy:  [0.69398148]
Loss:  22.328427612781525
Training accuracy:  [0.75532407]
Loss:  14.412934720516205
Training accuracy:  [0.87083333]
Loss:  10.425279274582863
Training accuracy:  [0.92662037]
Loss:  8.8801826313138
Training accuracy:  [0.94375]
Loss:  6.465670563280582
Training accuracy:  [0.9625]
Loss:  4.982062846422195
Training accuracy:  [0.98333333]
Loss:  3.6035365387797356
Training accuracy:  [0.99166667]
Loss:  3.101884040981531
Training accuracy:  [0.98958333]
Finished Training


In [322]:
print(calc_accuracy(net, test_x, test_y))

[0.58992806]


### Save features

In [157]:
def save_features(model, data_loader, outfile ):

    f = h5py.File(outfile, 'w')
    max_count = len(data_loader)*data_loader.batch_size
    all_labels = f.create_dataset('all_labels',(max_count,), dtype='i')
    all_feats=None
    count=0
    for i, (x,y) in enumerate(data_loader):
        #print(y)
        if i%10 == 0:
            print('{:d}/{:d}'.format(i, len(data_loader)))
        x_var = Variable(x)
        scores, feats = model(x_var)
        if all_feats is None:
            all_feats = f.create_dataset('all_feats', (max_count, feats.size(1)), dtype='f')
        all_feats[count:count+feats.size(0),:] = feats.data.cpu().numpy()
        all_labels[count:count+feats.size(0)] = y.cpu().numpy()
        #print(all_labels[count:count+feats.size(0)])
        count = count + feats.size(0)

    #print(all_labels)
    count_var = f.create_dataset('count', (1,), dtype='i')
    count_var[0] = count

    f.close()

In [ ]:
train_dataset = datasets.ImageFolder(root='../novel-data/train',
                                           transform=data_transform)
train_dataset_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size=20, shuffle=True,
                                             num_workers=4)
#pics, cls = next(iter(train_dataset_loader))
class_names = train_dataset.classes
#for x in cls:
#    print(x, class_names[x])
save_features(net, train_dataset_loader, 'resnet_features.hdf5')

In [141]:
base_classes = [0,6,7,8,9,10,11,12,13]
novel_classes = [1,2,3,4,5]

### K Means

In [143]:
def kmeans(x, k, niter=1, batchsize=1000):
    batchsize = min(batchsize, x.shape[0])

    nsamples = x.shape[0]
    ndims = x.shape[1]

    x2 = np.sum(x**2, axis=1)
    centroids = np.random.randn(k, ndims)
    centroidnorm = np.sqrt(np.sum(centroids**2, axis=1, keepdims=True))
    centroids = centroids / centroidnorm
    totalcounts = np.zeros(k)

    for i in range(niter):
        c2 = np.sum(centroids**2, axis=1,keepdims=True)*0.5
        summation = np.zeros((k, ndims))
        counts = np.zeros(k)
        loss = 0

        for j in range(0, nsamples, batchsize):
            lastj = min(j+batchsize, nsamples)
            batch = x[j:lastj]
            m = batch.shape[0]

            tmp = np.dot(centroids, batch.T)
            tmp = tmp - c2
            val = np.max(tmp,0)
            labels = np.argmax(tmp,0)
            loss = loss + np.sum(np.sum(x2[j:lastj])*0.5 - val)

            S = np.zeros((k, m))
            S[labels, np.arange(m)] = 1
            summation = summation + np.dot(S, batch)
            counts = counts + np.sum(S, axis=1)

        for j in range(k):
            if counts[j]>0:
                centroids[j] = summation[j] / counts[j]

        totalcounts = totalcounts + counts
        for j in range(k):
            if totalcounts[j] == 0:
                idx = np.random.choice(nsamples)
                centroids[j] = x[idx]



    return centroids

### Cluster features

In [144]:
def cluster_feats(filehandle, base_classes, cachefile, n_clusters=5):
    if os.path.isfile(cachefile):
        with open(cachefile, 'rb') as f:
            centroids = pickle.load(f)
    else:
        centroids = []
        all_labels = filehandle['all_labels'][...]
        all_feats = filehandle['all_feats']

        count = filehandle['count'][0]
        for j, i in enumerate(base_classes):
            print('Clustering class {:d}:{:d}'.format(j,i))
            idx = np.where(all_labels==i)[0]
            idx = idx[idx<count]
            X = all_feats[idx,:]
            # use a reimplementation of torch kmeans for reproducible results
            # TODO: Figure out why this is important
            centroids_this = kmeans(X, n_clusters, 20)
            centroids.append(centroids_this)
        with open(cachefile, 'wb') as f:
            pickle.dump(centroids, f)
    return centroids

In [145]:

with h5py.File('resnet_features.hdf5', 'r') as features_file:
    centroids = cluster_feats(features_file, base_classes, 'centroids.pkl', 5)

Clustering class 0:0
Clustering class 1:6
Clustering class 2:7
Clustering class 3:8
Clustering class 4:9
Clustering class 5:10
Clustering class 6:11
Clustering class 7:12
Clustering class 8:13


### Mine Analogies

In [146]:
def get_difference_vectors(c_i):
    diff_i = c_i[:,np.newaxis,:] - c_i[np.newaxis,:,:]
    diff_i = diff_i.reshape((-1, diff_i.shape[2]))
    diff_i_norm = np.sqrt(np.sum(diff_i**2,axis=1, keepdims=True))
    diff_i = diff_i / (diff_i_norm + 0.00001)
    return diff_i

def mine_analogies(centroids):
    n_clusters = centroids[0].shape[0]

    analogies = np.zeros((n_clusters*n_clusters*len(centroids),4), dtype=int)
    analogy_scores = np.zeros(analogies.shape[0])
    start=0

    I, J = np.unravel_index(np.arange(n_clusters**2), (n_clusters, n_clusters))
    # for every class
    for i, c_i in enumerate(centroids):

        # get normalized difference vectors between cluster centers
        diff_i = get_difference_vectors(c_i)
        diff_i_t = torch.Tensor(diff_i)


        bestdots = np.zeros(diff_i.shape[0])
        bestdotidx = np.zeros((diff_i.shape[0],2),dtype=int)

        # for every other class
        for j, c_j in enumerate(centroids):
            if i==j:
                continue
            print(i,j)

            # get normalized difference vectors
            diff_j = get_difference_vectors(c_j)
            diff_j = torch.Tensor(diff_j)

            #compute cosine distance and take the maximum
            dots = diff_i_t.mm(diff_j.transpose(0,1))
            maxdots, argmaxdots = dots.max(1)
            maxdots = maxdots.cpu().numpy().reshape(-1)
            argmaxdots = argmaxdots.cpu().numpy().reshape(-1)

            # if maximum is better than best seen so far, update
            betteridx = maxdots>bestdots
            bestdots[betteridx] = maxdots[betteridx]
            bestdotidx[betteridx,0] = j*n_clusters + I[argmaxdots[betteridx]]
            bestdotidx[betteridx,1] = j*n_clusters + J[argmaxdots[betteridx]]


        # store discovered analogies
        stop = start+diff_i.shape[0]
        analogies[start : stop,0]=i*n_clusters + I
        analogies[start : stop,1]=i*n_clusters + J
        analogies[start : stop,2:] = bestdotidx
        analogy_scores[start : stop] = bestdots
        start = stop

    #prune away trivial analogies
    good_analogies = (analogy_scores>0) & (analogies[:,0]!=analogies[:,1]) & (analogies[:,2]!=analogies[:,3])
    return analogies[good_analogies,:], analogy_scores[good_analogies]

In [147]:
analogies, analogy_scores = mine_analogies(centroids)
np.save('analogies.npy', analogies.astype(int))

0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
1 0
1 2
1 3
1 4
1 5
1 6
1 7
1 8
2 0
2 1
2 3
2 4
2 5
2 6
2 7
2 8
3 0
3 1
3 2
3 4
3 5
3 6
3 7
3 8
4 0
4 1
4 2
4 3
4 5
4 6
4 7
4 8
5 0
5 1
5 2
5 3
5 4
5 6
5 7
5 8
6 0
6 1
6 2
6 3
6 4
6 5
6 7
6 8
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 8
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7


### Train classifier

In [181]:
def train_classifier(filehandle, base_classes, cachefile, networkfile, total_num_classes=14, lr=0.1, wd=0.0001, momentum=0.9, batchsize=20, niter=500):
    # either use pre-existing classifier or train one
    all_labels = filehandle['all_labels'][...]
    all_labels = all_labels.astype(int)
    all_feats = filehandle['all_feats']
    base_class_ids = np.where(np.in1d(all_labels, base_classes))[0]
    #print(base_class_ids)
    loss = nn.CrossEntropyLoss()
    model = nn.Linear(all_feats[0].size, total_num_classes)
    if os.path.isfile(cachefile):
        tmp = torch.load(cachefile)
        model.load_state_dict(tmp)
    elif os.path.isfile(networkfile):
        tmp = torch.load(networkfile)
        if 'module.classifier.bias' in tmp['state']:
            state_dict = {'weight':tmp['state']['module.classifier.weight'], 'bias':tmp['state']['module.classifier.bias']}
        else:
            model = nn.Linear(all_feats[0].size, total_num_classes, bias=False).cuda()
            state_dict = {'weight':tmp['state']['module.classifier.weight']}
        model.load_state_dict(state_dict)
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr, momentum=momentum, weight_decay=wd, dampening=0)
        for i in range(niter):
            optimizer.zero_grad()
            idx = np.sort(np.random.choice(base_class_ids, batchsize, replace=False))
            F = all_feats[idx,:]
            F = Variable(torch.Tensor(F))
            L = Variable(torch.LongTensor(all_labels[idx]))
            S = model(F)
            loss_val = loss(S, L)
            loss_val.backward()
            optimizer.step()
            if i % 100 == 0:
                print('Classifier training {:d}: {:f}'.format(i, loss_val.data[0]))
        torch.save(model.state_dict(), cachefile)
        
    return model

In [242]:
with h5py.File('resnet_features.hdf5', 'r') as features_file:
    classification_model = train_classifier(features_file, base_classes, 'classifier.pkl', 'random.npy')

Classifier training 0: 2.660238
Classifier training 100: 0.047393
Classifier training 200: 0.039515
Classifier training 300: 0.003187
Classifier training 400: 0.024908


### Analogy regressor train

In [245]:
class AnalogyRegressor(nn.Module):
    def __init__(self, featdim, innerdim=512):
        super(AnalogyRegressor,self).__init__()
        self.featdim = featdim
        self.innerdim = innerdim
        self.fc1 = nn.Linear(featdim*3, innerdim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(innerdim, innerdim)
        self.fc3 = nn.Linear(innerdim, featdim)

    def forward(self, a,c,d):
        x = torch.cat((a,c,d), dim=1)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        return out

def train_analogy_regressor(analogies, centroids, base_classes, trained_classifier, lr=0.1, wt=10, niter=3000, step_after=5000, batchsize=40, momentum=0.9, wd=0.0001):
    # pre-permute analogies
    permuted_analogies = analogies[np.random.permutation(analogies.shape[0])]

    # create model and init
    featdim = centroids[0].shape[1]
    model = AnalogyRegressor(featdim)
    model = model
    trained_classifier = trained_classifier
    optimizer = torch.optim.SGD(model.parameters(), lr, momentum=momentum, weight_decay=wd, dampening=momentum)
    loss_1 = nn.CrossEntropyLoss()
    loss_2 = nn.MSELoss()


    num_clusters_per_class = centroids[0].shape[0]
    centroid_labels = (np.array(base_classes).reshape((-1,1)) * np.ones((1, num_clusters_per_class))).reshape(-1)
    concatenated_centroids = np.concatenate(centroids, axis=0)


    start=0
    avg_loss_1 = avg_loss_2 = count = 0.0
    for i in range(niter):
        # get current batch of analogies
        stop = min(start+batchsize, permuted_analogies.shape[0])
        #print(start+batchsize, permuted_analogies.shape[0])
        to_train = permuted_analogies[start:stop,:]
        optimizer.zero_grad()

        # analogy is A:B :: C:D, goal is to predict B from A, C, D
        # Y is the class label of B (and A)
        A = concatenated_centroids[to_train[:,0]]
        B = concatenated_centroids[to_train[:,1]]
        C = concatenated_centroids[to_train[:,2]]
        D = concatenated_centroids[to_train[:,3]]
        Y = centroid_labels[to_train[:,1]]

        A = Variable(torch.Tensor(A))
        B = Variable(torch.Tensor(B))
        C = Variable(torch.Tensor(C))
        D = Variable(torch.Tensor(D))
        Y = Variable(torch.LongTensor(Y.astype(int)))

        Bhat = model(A,C,D)

        lossval_2 = loss_2(Bhat, B) # simple mean squared error loss

        # classification loss
        predicted_classprobs = trained_classifier(Bhat)
        lossval_1 = loss_1(predicted_classprobs, Y)
        loss = lossval_1 + wt * lossval_2

        loss.backward()
        optimizer.step()

        avg_loss_1 = avg_loss_1 + lossval_1.data[0]
        avg_loss_2 = avg_loss_2 + lossval_2.data[0]
        count = count+1.0


        if i % 50 == 0:
            print('{:d} : {:f}, {:f}, {:f}'.format(i, avg_loss_1/count, avg_loss_2/count, count))
            avg_loss_1 = avg_loss_2 = count = 0.0

        if (i+1) % step_after == 0:
            lr = lr / 10.0
            print(lr)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

        start = stop
        if start==permuted_analogies.shape[0]:
            start=0

    return dict(model_state=model.state_dict(), concatenated_centroids=torch.Tensor(concatenated_centroids),
            num_base_classes=len(centroids), num_clusters_per_class=num_clusters_per_class)

In [243]:
print(analogies.shape[0])

180


In [246]:
generator = train_analogy_regressor(analogies, centroids, base_classes, classification_model, lr=0.1)

0 : 3.957476, 0.454064, 1.000000
50 : 4.274689, 0.423214, 50.000000
100 : 0.962870, 0.244093, 50.000000
150 : 0.237396, 0.162183, 50.000000
200 : 0.001571, 0.111788, 50.000000
250 : 0.000898, 0.096564, 50.000000
300 : 0.000764, 0.089482, 50.000000
350 : 0.000681, 0.084209, 50.000000
400 : 0.000541, 0.081249, 50.000000
450 : 0.000435, 0.079073, 50.000000
500 : 0.000378, 0.077413, 50.000000
550 : 0.000344, 0.075729, 50.000000
600 : 0.000306, 0.073976, 50.000000
650 : 0.000280, 0.072458, 50.000000
700 : 0.000248, 0.071526, 50.000000
750 : 0.000226, 0.070863, 50.000000
800 : 0.000221, 0.070271, 50.000000
850 : 0.000204, 0.069605, 50.000000
900 : 0.000189, 0.068854, 50.000000
950 : 0.000183, 0.068201, 50.000000
1000 : 0.000175, 0.067466, 50.000000
1050 : 0.000179, 0.066125, 50.000000
1100 : 0.000170, 0.065601, 50.000000
1150 : 0.000161, 0.065168, 50.000000
1200 : 0.000154, 0.064760, 50.000000
1250 : 0.000147, 0.064369, 50.000000
1300 : 0.000141, 0.063993, 50.000000
1350 : 0.000134, 0.063635

In [247]:
print(generator['num_base_classes'])
print(generator['concatenated_centroids'].size(1))
#torch.save(generator, open('generator.tar','wb'))

9
512


### Generate

In [248]:
def init_generator():
#     G = torch.load(open('generator.tar'), 'rb')
    featdim = generator['concatenated_centroids'].size(1)
    model = AnalogyRegressor(featdim)
    model.load_state_dict(generator['model_state'])
#    G['model'] =model
    return model

In [250]:
def do_generate(feats, labels, generator, gen_model, max_per_label):
    # generate till there are at least max_per_label examples for each label
    unique_labels = np.unique(labels)
    print(unique_labels)
    generations_needed = []
    generator['concatenated_centroids'] = generator['concatenated_centroids']
    for k, lab in enumerate(unique_labels):
        # for each label
        idx = np.where(labels==lab)[0]
        print(idx.size)
        # generate this many examples:
        num_to_gen = max(max_per_label - idx.size,0)
        
        if num_to_gen>0:
            # choose a random seed
            seed = np.random.choice(idx, num_to_gen)
            # and a random base class
            base_class = np.random.choice(generator['num_base_classes'], num_to_gen)
            # and two random centroids from this base class
            c_c = np.random.choice(generator['num_clusters_per_class'], num_to_gen)
            c_d = np.random.choice(generator['num_clusters_per_class'], num_to_gen)

            centroid_ids_c = base_class*generator['num_clusters_per_class'] + c_c
            centroid_ids_d = base_class*generator['num_clusters_per_class'] + c_d
            # add to list of things to generate
            generations_needed.append( np.concatenate((seed.reshape((-1,1)), centroid_ids_c.reshape((-1,1)), centroid_ids_d.reshape((-1,1))),axis=1))

    if len(generations_needed)>0:
        generations_needed = np.concatenate(generations_needed, axis=0)
        gen_feats = np.zeros((generations_needed.shape[0],feats.shape[1]))
        gen_labels = np.zeros(generations_needed.shape[0])


        # batch up the generations
        batchsize=20
        for start in range(0, generations_needed.shape[0], batchsize):
            stop = min(start + batchsize, generations_needed.shape[0])
            g_idx = generations_needed[start:stop,:]
            A = Variable(torch.Tensor(feats[g_idx[:,0],:]))
            C = Variable(torch.Tensor(generator['concatenated_centroids'][g_idx[:,1],:]))
            D = Variable(torch.Tensor(generator['concatenated_centroids'][g_idx[:,2],:]))
            F = gen_model(A,C,D).cpu().data.numpy().copy()
            gen_feats[start:stop,:] = F
            print(np.linalg.norm(F-feats[g_idx[:,0],:]), np.linalg.norm(F), np.linalg.norm(feats[g_idx[:,0],:]))
            gen_labels[start:stop] = labels[g_idx[:,0]]

        return np.concatenate((feats, gen_feats), axis=0), np.concatenate((labels, gen_labels), axis=0)
    else:
        return feats, labels

In [251]:
with h5py.File('resnet_features.hdf5', 'r') as filehandle:
    all_labels = filehandle['all_labels'][...]
    #all_labels = all_labels.astype(int)
    all_feats = filehandle['all_feats']
    novel_class_ids = np.where(np.in1d(all_labels, novel_classes))[0]
    novel_features = all_feats[novel_class_ids, :]
    novel_labels = all_labels[novel_class_ids]
    gen_model = init_generator()
    gen_features, gen_labels = do_generate(novel_features, novel_labels, generator, gen_model, 50)

[1 2 3 4 5]
5
5
5
5
5
37.71373 64.06834 72.00206
39.251198 59.015026 75.34595
32.94492 60.190857 66.40843
31.743113 58.34321 62.085266
33.535183 60.09323 64.57061
32.890892 58.827156 65.2942
36.558365 57.91354 71.832565
36.727913 62.4228 71.87672
38.65991 62.796574 72.86085
33.7015 64.69294 70.049774
35.24881 63.1426 68.47793
17.64901 31.978945 33.336136


### Test

In [290]:
test_dataset = datasets.ImageFolder(root='../novel-data/test',
                                           transform=data_transform)
test_dataset_loader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size=20, shuffle=True,
                                             num_workers=4)
#pics, cls = next(iter(test_dataset_loader))
#class_names = test_dataset.classes
#for x in cls:
#    print(x, class_names[x])
save_features(net, test_dataset_loader, 'resnet_features_test.hdf5')

0/7


In [294]:
def calc_class_accuracy(mdl, X, Y):
    # TODO: why can't we call .data.numpy() for train_acc as a whole?
    outputs = mdl(X)
    max_vals, max_indices = torch.max(outputs,1)
    acc = (max_indices == Y).sum().data.numpy()/max_indices.size()[0]
    print(max_indices, Y)
    return acc

In [ ]:
with h5py.File('resnet_features.hdf5', 'r') as filehandle:
    all_labels = filehandle['all_labels'][...]
    all_labels = all_labels.astype(int)
    all_feats = filehandle['all_feats']
    base_class_ids = np.where(np.in1d(all_labels, base_classes))[0]
    novel_class_ids = np.where(np.in1d(all_labels, novel_classes))[0]
    print(novel_class_ids)
    idx = np.sort(np.random.choice(novel_class_ids, 25, replace=False))
    F = all_feats[idx,:]
    F = Variable(torch.Tensor(F))
    L = Variable(torch.LongTensor(all_labels[idx]))
    print(calc_class_accuracy(classification_model, F, L))

In [299]:
def train_base_novel_classifier(features, labels, lr=0.1, wd=0.0001, momentum=0.9):
    loss = nn.CrossEntropyLoss()
    model = nn.Linear(features[0].size, 6)
    optimizer = torch.optim.SGD(model.parameters(), lr, momentum=momentum, weight_decay=wd, dampening=0)
    for i in range(500):
        optimizer.zero_grad()
        idx = np.sort(np.random.choice(np.arange(250), 20, replace=False))
        #print(idx)
        F = features[idx,:]
        F = Variable(torch.Tensor(features))
        L = Variable(torch.LongTensor(labels))
        S = model(F)
        #print(calc_class_accuracy(model, F, L))
        loss_val = loss(S, L)
        loss_val.backward()
        optimizer.step()
        
    return model

In [300]:
novel_classifier = train_base_novel_classifier(gen_features, gen_labels)
#print(gen_labels.shape)
#print(calc_class_accuracy(classification_model, F, L))

In [305]:
with h5py.File('resnet_features_test.hdf5', 'r') as filehandle:
    all_labels = filehandle['all_labels'][...]
    all_labels = all_labels.astype(int)
    all_feats = filehandle['all_feats']
    novel_class_ids = np.where(np.in1d(all_labels, novel_classes))[0]
    print(novel_class_ids)
    idx = np.sort(np.random.choice(novel_class_ids, 25, replace=False))
    F = all_feats[idx,:]
    F = Variable(torch.Tensor(F))
    L = Variable(torch.LongTensor(all_labels[idx]))
    print(calc_class_accuracy(novel_classifier, F, L))

[  0   1   7   8  12  16  20  21  22  25  29  32  37  38  39  42  47  54
  55  57  58  62  63  64  67  72  75  78  89  90  93  96  98 100 103 108
 109 110 111 112 117 120 122 124 125 128 129 136 138]
Variable containing:
 2
 1
 2
 2
 3
 4
 1
 4
 2
 1
 3
 4
 3
 4
 4
 4
 5
 1
 2
 4
 4
 4
 2
 4
 2
[torch.LongTensor of size 25]
 Variable containing:
 3
 4
 5
 4
 1
 4
 3
 4
 1
 3
 5
 3
 5
 2
 3
 1
 5
 3
 3
 1
 2
 4
 3
 2
 1
[torch.LongTensor of size 25]

[0.16]
